# Using SQL in Python

In [ ]:
#installing packages
!pip install -q -U pandas
!pip install -q ipython-sql


In [1]:
#importing packages
import pandas as pd
import sqlite3
import requests
import io

In [2]:
#importing the dataset from Github repository

url = 'https://raw.githubusercontent.com/manthayconte/python_for_data_analysis/main/exercise_1/dataset/diabetes.csv'
download = requests.get(url).content

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [3]:
#checking the dataset basic data
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
#the dataset has 768 observations and 9 variables
df.shape

(768, 9)

In [ ]:
#if you want to download the database from GitHub:
#url = 'https://github.com/manthayconte/python_for_data_analysis/tree/main/exercise_1/database/dbprojeto1.db'
#urllib.request.urlretrieve(url, 'dbprojeto1.db')

In [5]:
#creating a connection to a SQLite database
cnn = sqlite3.connect('/Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db')

In [6]:
#write the dataFrame to the database as a table
df.to_sql('diabetes', cnn, if_exists = 'replace')

768

In [7]:
#loading the SQL extension 
%load_ext sql

In [17]:
#defining the database
%sql sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db


In [19]:
#checking the observations number
%%sql

SELECT count(*) as contagem

FROM diabetes

 * sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db
   sqlite:///database/dbprojeto1.db
Done.


contagem
768


In [34]:
%%sql

SELECT * FROM diabetes LIMIT 5

 * sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db
   sqlite:///database/dbprojeto1.db
Done.


index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [40]:
#adding a column to categorize the Age 
%%sql

ALTER TABLE diabetes
ADD age_group VARCHAR(20)

 * sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db
   sqlite:///database/dbprojeto1.db
Done.


[]

In [42]:
#adding the categorized information in age_group
%%sql
UPDATE diabetes
SET age_group = (CASE
    WHEN Age < 20 THEN 'LESS THEN 20'
    WHEN Age >= 20 AND Age < 30 THEN 'BETWEEN 20 AND 30'
    WHEN Age >= 30 AND Age < 40 THEN 'BETWEEN 30 AND 40'
    WHEN Age >= 40 AND Age <= 50 THEN 'BETWEEN 40 AND 50'
    WHEN Age > 50 THEN 'MORE THEN 50'
  END)

 * sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db
   sqlite:///database/dbprojeto1.db
768 rows affected.


[]

In [44]:
#checking the new column
%%sql

SELECT * FROM diabetes LIMIT(5)

 * sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db
   sqlite:///database/dbprojeto1.db
Done.


index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,age_group
0,6,148,72,35,0,33.6,0.627,50,1,BETWEEN 40 AND 50
1,1,85,66,29,0,26.6,0.351,31,0,BETWEEN 30 AND 40
2,8,183,64,0,0,23.3,0.672,32,1,BETWEEN 30 AND 40
3,1,89,66,23,94,28.1,0.167,21,0,BETWEEN 20 AND 30
4,0,137,40,35,168,43.1,2.288,33,1,BETWEEN 30 AND 40


In [73]:
#descriptive analysis using categorized age_group
%%sql
SELECT
    CASE WHEN Outcome = 0 THEN 'NO' ELSE 'YES' END as 'diabetes',
    age_group,
    ROUND(AVG(Glucose),0) as 'AVG_glucose',
    MAX(Glucose) as 'MAX_glucose',
    MIN(Glucose) as 'MIN_glucose',
    ROUND(AVG(BloodPressure),0) as 'AVG_bloodPressure',
    MAX(BloodPressure) as 'MAX_bloodPressure',
    MIN(BloodPressure) as 'MIN_bloodPressure',
    ROUND(AVG(BMI),0) as 'AVG_bmi',
    MAX(BMI) as 'MAX_bmi',
    MIN(BMI) as 'MIN_bmi'
FROM diabetes 
GROUP BY age_group, Outcome

 * sqlite:////Users/gabrielconte/Documents/GitHub/Python_data_analytics/exercise_1/database/dbprojeto1.db
   sqlite:///database/dbprojeto1.db
Done.


diabetes,age_group,AVG_glucose,MAX_glucose,MIN_glucose,AVG_bloodPressure,MAX_bloodPressure,MIN_bloodPressure,AVG_bmi,MAX_bmi,MIN_bmi
NO,BETWEEN 20 AND 30,107.0,193,0,65.0,122,0,30.0,57.3,0.0
YES,BETWEEN 20 AND 30,141.0,199,78,65.0,110,0,37.0,67.1,22.9
NO,BETWEEN 30 AND 40,113.0,197,44,71.0,110,0,31.0,46.8,19.3
YES,BETWEEN 30 AND 40,139.0,197,0,68.0,108,0,34.0,50.0,0.0
NO,BETWEEN 40 AND 50,111.0,183,57,74.0,106,0,33.0,46.7,20.8
YES,BETWEEN 40 AND 50,137.0,196,0,75.0,114,0,35.0,52.3,24.3
NO,MORE THEN 50,127.0,194,57,76.0,108,0,28.0,46.5,0.0
YES,MORE THEN 50,154.0,197,90,81.0,110,60,32.0,45.4,0.0


In [75]:
# Results show that glucose average is a significant factor in determining diabetes across all age groups.

# The blood pressure doesn't appear to play a significant role in determining diabetes in the analyzed data.

# The BMI factor has more significance in determining diabetes in people aged 20-30, but outliers like zero BMI should be checked.